In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import numpy


# Define transform to convert to tensor and normalize the data
# with mean and std = 0.5
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load Fashion MNIST dataset
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                             download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                            download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=20,
                                         num_workers=6,
                                         shuffle=False,
                                         drop_last=True)

In [2]:
print(trainset[1][0].shape)
print(trainset[1][1])
print(trainset)
print(len(trainset[0]))
print(type(trainset[0][1]))
print(trainset[30000][0][0])

torch.Size([1, 28, 28])
0
Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )
2
<class 'int'>
tensor([[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -0.0745,  0.6000,  0.4196,  0.3725,  0.6706,  0.5608,
          0.3176,  0.5451, -0.1294, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000,  0.3569,  0.7647,  0.4510,  0.4039,  0.7647,  0.2392,
          0.1137,  0.7804,  0.3569, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000,  0.3333,  0.7961,  0.7725,  0.7725,  0.8275, 

In [3]:
in_channels = 1
hidden_layers = [(64, 3), (64, 3)]  # (output weights, kernel size)
activation = 'ReLU'
pooling = 'max'
optimizer = 'adam'
lr = 0.001
dropout_prob = 0.5
epochs = 5
IMAGE_SIZE = 28

def build_cnn(in_channels,
               hidden_layers,
               activation,
               pooling,
               optimizer,
               lr,
               dropout_prob=0
               ):
    layers = []

    # Define input layer 
    layers.append(nn.Conv2d(in_channels=in_channels,
                            out_channels=IMAGE_SIZE,
                            kernel_size=3,
                            padding=1
                            ))
    
    prev_channels = IMAGE_SIZE
    for filters, kernel_size in hidden_layers:
        layers.append(nn.Conv2d(in_channels=prev_channels,
                                out_channels=filters,
                                kernel_size=kernel_size,
                                padding=1))
        layers.append(getattr(nn, activation)())

        if pooling == 'max':
            layers.append(nn.MaxPool2d(2, 2))
        elif pooling == 'avg':
            layers.append(nn.AvgPool2d(2, 2))
        else:
            raise ValueError(f'Invalid pooling function: {pooling}')
        
        if dropout_prob > 0:
            layers.append(nn.Dropout(dropout_prob))
        
        prev_channels = filters

    # Flatten for fully connected layers
    layers.append(nn.Flatten())

    layers.append(nn.Linear(in_features=prev_channels * 7 * 7,
                            out_features=64))
    layers.append(getattr(nn, activation)())

    layers.append(nn.Linear(64, 10))

    model = nn.Sequential(*layers)

    if optimizer == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    elif optimizer == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    elif optimizer == 'rmsprop':
        optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
    else:
        raise ValueError(f'Invalid optimizer: {optimizer}')
    
    return model, optimizer

In [4]:
# Define training and evaluation function
def train_and_evaluate(model, optimizer, train_loader, test_loader, epochs):
  """
  Trains the model and evaluates performance on the test set.

  Args:
      model: The CNN model to train.
      optimizer: The optimizer to use.
      train_loader: DataLoader for training data.
      test_loader: DataLoader for test data.
      epochs: Number of training epochs.

  Returns:
      The average test accuracy over epochs.
  """

  criterion = nn.CrossEntropyLoss()  # Loss function

  for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
      inputs, labels = data
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item()
      if i % 2000 == 1999:
        print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

  print('Finished Training')

  # Evaluation on test set
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for data in test_loader:
      images, labels = data
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))
  return 100 * correct / total

In [5]:
# Define parameters for CNN model
in_channels = 1
hidden_layers = [(64, 3), (64, 3)]  # Example hidden layers
activation = 'ReLU'  # Corrected activation function name
pooling = 'max'
optimizer = 'adam'
lr = 0.001
dropout_prob = 0.5
epochs = 5

# Build the CNN model
model, optimizer = build_cnn(in_channels, hidden_layers, activation, pooling, optimizer, lr, dropout_prob)

# Train and evaluate the model
test_accuracy = train_and_evaluate(model, optimizer, trainloader, trainloader, epochs)

[1,  2000] loss: 0.745
[1,  4000] loss: 0.547
[1,  6000] loss: 0.489
[1,  8000] loss: 0.475
[1, 10000] loss: 0.444
[1, 12000] loss: 0.453
[1, 14000] loss: 0.447
[2,  2000] loss: 0.418
[2,  4000] loss: 0.402
[2,  6000] loss: 0.414
[2,  8000] loss: 0.395
[2, 10000] loss: 0.411
[2, 12000] loss: 0.402
[2, 14000] loss: 0.406
[3,  2000] loss: 0.388
[3,  4000] loss: 0.408
[3,  6000] loss: 0.389
[3,  8000] loss: 0.373
[3, 10000] loss: 0.377
[3, 12000] loss: 0.373
[3, 14000] loss: 0.380
[4,  2000] loss: 0.381
[4,  4000] loss: 0.383
[4,  6000] loss: 0.382
[4,  8000] loss: 0.376
[4, 10000] loss: 0.363
[4, 12000] loss: 0.374
[4, 14000] loss: 0.394
[5,  2000] loss: 0.377
[5,  4000] loss: 0.358
[5,  6000] loss: 0.377
[5,  8000] loss: 0.356
[5, 10000] loss: 0.374
[5, 12000] loss: 0.383
[5, 14000] loss: 0.365
Finished Training
Accuracy of the network on the test images: 90 %


In [10]:
# Define parameters for CNN model
in_channels = 1
hidden_layers = [(64, 3), (64, 3)]  # Example hidden layers
activation = 'ReLU'  # Corrected activation function name
pooling = 'max'
optimizer = 'adam'
lr = 0.005
dropout_prob = 0
epochs = 5

# Build the CNN model
model, optimizer = build_cnn(in_channels, hidden_layers, activation, pooling, optimizer, lr, dropout_prob)
# devide = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(devide)

# Train and evaluate the model
test_accuracy = train_and_evaluate(model, optimizer, trainloader, trainloader, epochs)

[1,  2000] loss: 0.879
[1,  4000] loss: 0.799
[1,  6000] loss: 0.645
[1,  8000] loss: 0.613
[1, 10000] loss: 0.650
[1, 12000] loss: 0.560
[1, 14000] loss: 0.560
[2,  2000] loss: 0.628
[2,  4000] loss: 0.506
[2,  6000] loss: 0.634
[2,  8000] loss: 0.556
[2, 10000] loss: 0.502
[2, 12000] loss: 0.639
[2, 14000] loss: 0.522
[3,  2000] loss: 0.517
[3,  4000] loss: 0.530
[3,  6000] loss: 0.496
[3,  8000] loss: 0.529
[3, 10000] loss: 0.508
[3, 12000] loss: 0.716
[3, 14000] loss: 0.535
[4,  2000] loss: 0.541
[4,  4000] loss: 0.496
[4,  6000] loss: 1.012
[4,  8000] loss: 2.308
[4, 10000] loss: 2.304
[4, 12000] loss: 2.305
[4, 14000] loss: 2.304
[5,  2000] loss: 2.305
[5,  4000] loss: 2.305
[5,  6000] loss: 2.304
[5,  8000] loss: 2.305
[5, 10000] loss: 2.304
[5, 12000] loss: 2.305
[5, 14000] loss: 2.304
Finished Training
Accuracy of the network on the test images: 10 %
